In [ ]:
# File preparation

In [134]:
# upload file (.pdf,.docx,.txt,.xlsx)
import os
import fitz  # PyMuPDF (für PDF)
import chardet
from docx import Document
from IPython.display import display
import ipywidgets as widgets
import pandas as pd

def excel_to_txt(file_content, filename):
    df = pd.read_excel(file_content, engine='openpyxl')
    output_filename = filename.replace('.xlsx', '.txt').replace('.xls', '.txt')
    df.to_csv(output_filename, sep='\t', index=False)
    return output_filename

upload = widgets.FileUpload(accept='.pdf,.docx,.txt,.xlsx,.xls', multiple=False)
display(upload)

def on_upload_change(change):
    for filename, file_info in upload.value.items():
        file_content = file_info['content']
        if filename.lower().endswith('.xlsx') or filename.lower().endswith('.xls'):
            with open('temp.xlsx', 'wb') as f:
                f.write(file_content)
            output_filename = excel_to_txt('temp.xlsx', filename)
            print(f"Excel-Datei wurde in {output_filename} umgewandelt!")
        elif filename.lower().endswith('.docx'):
            # DOCX-Logik hier einfügen
            pass
        elif filename.lower().endswith('.pdf'):
            # PDF-Logik hier einfügen
            pass
        elif filename.lower().endswith('.txt'):
            # TXT-Logik hier einfügen
            pass

upload.observe(on_upload_change, names='value')


FileUpload(value=(), accept='.pdf,.docx,.txt,.xlsx,.xls', description='Upload')

In [155]:
# Upload-Daten korrekt auslesen (auch in neueren Jupyter-Versionen)
if isinstance(upload.value, dict):  # alte Struktur
    uploaded_file = next(iter(upload.value.items()))
    uploaded_filename = uploaded_file[0]
    uploaded_bytes = uploaded_file[1]["content"]
elif isinstance(upload.value, tuple) and len(upload.value) > 0:  # neue Struktur
    uploaded_file = upload.value[0]
    uploaded_filename = uploaded_file["name"]
    uploaded_bytes = uploaded_file["content"]
else:
    raise ValueError("⚠️ Keine Datei hochgeladen.")

# Datei speichern
with open(uploaded_filename, "wb") as f:
    f.write(uploaded_bytes)
def convert_to_txt(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".txt":
        return file_path
    elif ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join([page.get_text() for page in doc])
    elif ext == ".docx":
        doc = Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    elif ext in (".xlsx", ".xls"):
        # Excel-Datei einlesen
        df = pd.read_excel(file_path, engine='openpyxl')
        # Als TXT (Tab-getrennt) speichern
        txt_path = os.path.splitext(file_path)[0] + ".txt"
        df.to_csv(txt_path, sep='\t', index=False, encoding='utf-8')
        return txt_path
    else:
        raise ValueError("❌ Nur PDF, DOCX, TXT oder XLS/XLSX erlaubt.")

    txt_path = os.path.splitext(file_path)[0] + ".txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)
    return txt_path

# Hauptvariable setzen
DOCUMENT_NAME = convert_to_txt(uploaded_filename)
print(f"📄 Verwendete Textdatei: {DOCUMENT_NAME}")

📄 Verwendete Textdatei: 162853_CP Liban_FR.txt


In [156]:
# Step 1: Word Counter

import chardet
import re
import csv
from collections import defaultdict
import os  # <-- NEW: for filename handling

# === Konfigurierbarer Dateiname ===
#DOCUMENT_NAME = "162514_Focus Afrika_EN.txt"  # <--- Nur hier anpassen!

# === Schritt 1: Sachgebiet-CSV einlesen ===
sachgebiete = defaultdict(list)

with open("ED_Pub_Sachgebiet_multilingual5.csv", "r", encoding="utf-8") as file:
    reader = csv.DictReader(file, delimiter=";")
    for row in reader:
        sachgebiet_roh = row["Header.Sachgebiete"]
        sachgebiet_liste = [s.strip() for s in sachgebiet_roh.split(";")]
        for sachgebiet in sachgebiet_liste:
            for key in row.keys():
                if key.startswith(("DES", "ENG", "FRS", "ITS")) and row[key]:
                    sachgebiete[sachgebiet].append(row[key])

print(f"✅ Sachgebiete aus CSV geladen ({len(sachgebiete)} Sachgebiete)\n")

# === Schritt 2: Encoding automatisch erkennen ===
def detect_encoding(dateipfad):
    with open(dateipfad, "rb") as f:
        rawdata = f.read(10000)
        result = chardet.detect(rawdata)
        return result["encoding"]

# === Schritt 3: Neuen Text einlesen ===
dateipfad = DOCUMENT_NAME
encoding = detect_encoding(dateipfad)
print(f"🔎 Erkanntes Encoding: {encoding}")

with open(dateipfad, "r", encoding=encoding) as file:
    neuer_text = file.read()

print(f"✅ Textdatei '{dateipfad}' erfolgreich eingelesen ({len(neuer_text)} Zeichen)\n")

# === Schritt 4: Termini zählen
sachgebiet_counts = defaultdict(int)
gefundene_terme = defaultdict(lambda: defaultdict(int))

for sachgebiet, terme in sachgebiete.items():
    for term in set(terme):
        pattern = rf"(?<!\w){re.escape(term)}(?!\w)"
        matches = re.findall(pattern, neuer_text, re.IGNORECASE)
        count = len(matches)

        if count > 0:
            sachgebiet_counts[sachgebiet] += count
            gefundene_terme[sachgebiet][term] += count

print("✅ Termini-Zählung abgeschlossen.\n")

# === Schritt 5: Ergebnisse anzeigen
print("📊 Gefundene Termini je Sachgebiet:\n")

for sachgebiet, terme in gefundene_terme.items():
    print(f"🗂️  {sachgebiet}:")
    for term, count in terme.items():
        print(f"   - {term}: {count}")
    print()

# === Schritt 6: Top 3 Sachgebiete bestimmen
top_sachgebiete = sorted(sachgebiet_counts.items(), key=lambda x: x[1], reverse=True)[:3]
top_sachgebiete_namen = [sg for sg, _ in top_sachgebiete]

while len(top_sachgebiete_namen) < 3:
    top_sachgebiete_namen.append("")

print(f"🏅 Top 3 Sachgebiete: {top_sachgebiete_namen}\n")

# === Schritt 7: Ergebnis in eine CSV schreiben
# Automatisch den Output-Dateinamen erzeugen
basename, _ = os.path.splitext(DOCUMENT_NAME)
csv_dateiname = f"{basename}_top3.csv"

with open(csv_dateiname, mode="w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Dateiname", "Text", "Top-Sachgebiet 1", "Top-Sachgebiet 2", "Top-Sachgebiet 3"])
    writer.writerow([dateipfad, neuer_text] + top_sachgebiete_namen)

print(f"✅ CSV-Datei '{csv_dateiname}' wurde erfolgreich erstellt.\n")

✅ Sachgebiete aus CSV geladen (31 Sachgebiete)

🔎 Erkanntes Encoding: utf-8
✅ Textdatei '162853_CP Liban_FR.txt' erfolgreich eingelesen (2616 Zeichen)

✅ Termini-Zählung abgeschlossen.

📊 Gefundene Termini je Sachgebiet:

🗂️  Diplomatie und Protokoll:
   - communiqué de presse: 1

🗂️  HR:
   - spécialiste: 1

🗂️  Struktur und Institution:
   - Direction du développement et de la coopération: 1
   - domaine: 1
   - Aide humanitaire: 1
   - direction: 1

🗂️  Entwicklung und Zusammenarbeit:
   - Direction du développement et de la coopération: 1
   - direction: 1

🗂️  Humanitäre Hilfe:
   - Aide humanitaire: 1
   - aide humanitaire: 1

🗂️  Geschäftsverwaltung:
   - destruction: 1

🗂️  Buchhaltung:
   - projet: 1

🏅 Top 3 Sachgebiete: ['Struktur und Institution', 'Entwicklung und Zusammenarbeit', 'Humanitäre Hilfe']

✅ CSV-Datei '162853_CP Liban_FR_top3.csv' wurde erfolgreich erstellt.



In [157]:
# Step 2: Chunking
# 1. CSV einlesen und vorbereiten

import pandas as pd
from transformers import AutoTokenizer

# Lade den BERT-Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Lade das neue CSV
df_new_text = pd.read_csv(csv_dateiname)

# Spalten für das Chunking beibehalten
metadata_cols = ["Dateiname", "Top-Sachgebiet 1", "Top-Sachgebiet 2", "Top-Sachgebiet 3"]

# Store all chunked rows
expanded_rows = []


In [158]:
# 2. Chunking der Texte
# Neuen Basename aus der CSV erzeugen
basename, _ = os.path.splitext(csv_dateiname)
chunks_csv_name = f"{basename}_chunks.csv"
                                  
# Hilfsfunktion zum Chunken von Texten
def chunk_text(text, max_len=512):
    # Zerlege den Text in Token
    tokens = tokenizer.tokenize(text)
    
    # Überprüfe, ob der Text zu lang ist, und chunk ihn dann in 512-Tokens
    for i in range(0, len(tokens), max_len):
        chunk_tokens = tokens[i:i + max_len]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
        yield chunk_text

# Iteriere durch jede Zeile im DataFrame
for idx, row in df_new_text.iterrows():
    text = row["Text"]
    
    # Chunken des Textes in Segmente von max. 512 Tokens
    for chunk_id, chunk_text in enumerate(chunk_text(text), start=1):
        new_row = {col: row[col] for col in metadata_cols}
        new_row["Text"] = chunk_text
        new_row["original_row_id"] = idx
        new_row["chunk_id"] = chunk_id
        expanded_rows.append(new_row)

# Neue DataFrame erstellen
expanded_df = pd.DataFrame(expanded_rows)

# Optional: Speichern

expanded_df.to_csv(chunks_csv_name, index=False, encoding="utf-8")
print(f"✅ Chunked CSV '{chunks_csv_name}' wurde erfolgreich erstellt.\n")


Token indices sequence length is longer than the specified maximum sequence length for this model (961 > 512). Running this sequence through the model will result in indexing errors


✅ Chunked CSV '162853_CP Liban_FR_top3_chunks.csv' wurde erfolgreich erstellt.



In [160]:
# 3. Hinzufügen der "Top-3 alle"-Spalte
# Neuen Basename aus der CSV erzeugen
basename, _ = os.path.splitext(chunks_csv_name)
done_csv_name = f"{basename}_done.csv"

# Lade die CSV mit den Chunked-Daten
df_chunked = pd.read_csv(chunks_csv_name)

# Erstelle die neue Spalte "Top-3 alle" mit dem gewünschten Satz
df_chunked["Top-3 alle"] = (
    "Die Top 3 Sachgebiete in diesem Text sind: " +
    df_chunked["Top-Sachgebiet 1"].astype(str) + ", " +
    df_chunked["Top-Sachgebiet 2"].astype(str) + ", " +
    df_chunked["Top-Sachgebiet 3"].astype(str) + "."
)

# Spalten neu ordnen, um "Top-3 alle" zwischen "Top-Sachgebiet 3" und "Text" zu setzen
cols = df_chunked.columns.tolist()

# Finden des Index der Spalte "Text"
insert_at = cols.index("Text") if "Text" in cols else len(cols)

# Entferne "Top-3 alle" von der letzten Stelle (falls es an irgendeiner Stelle zugefügt wurde)
if "Top-3 alle" in cols:
    cols.remove("Top-3 alle")

# Füge "Top-3 alle" an der richtigen Stelle zwischen "Top-Sachgebiet 3" und "Text" ein
cols = cols[:insert_at] + ["Top-3 alle"] + cols[insert_at:]

# Reorganisiere den DataFrame mit der neuen Spaltenreihenfolge
df_chunked = df_chunked[cols]

# Speichere das neue CSV
df_chunked.to_csv(f"{basename}_done.csv", index=False)
print(f"✅ Done CSV '{done_csv_name}' wurde erfolgreich erstellt.\n")


✅ Done CSV '162853_CP Liban_FR_top3_chunks_done.csv' wurde erfolgreich erstellt.



In [161]:
# 4. Überprüfen des Ergebnisses
basename, _ = os.path.splitext(chunks_csv_name)
done_csv_name = f"{basename}_done.csv"

# Lade das neue CSV
df_new = pd.read_csv(done_csv_name)

# Zeige die ersten 3 Zeilen für eine Vorschau
print(df_new.head())

# Zeige die Struktur der Datei
print(df_new.shape)  # Zeilen und Spalten
print(df_new.columns.tolist())  # Spaltennamen


                Dateiname          Top-Sachgebiet 1  \
0  162853_CP Liban_FR.txt  Struktur und Institution   
1  162853_CP Liban_FR.txt  Struktur und Institution   

                 Top-Sachgebiet 2  Top-Sachgebiet 3  \
0  Entwicklung und Zusammenarbeit  Humanitäre Hilfe   
1  Entwicklung und Zusammenarbeit  Humanitäre Hilfe   

                                          Top-3 alle  \
0  Die Top 3 Sachgebiete in diesem Text sind: Str...   
1  Die Top 3 Sachgebiete in diesem Text sind: Str...   

                                                Text  original_row_id  \
0  Communiqué de presse Berne 09. 12. 2024 ( TBC ...                0   
1  ##ès équitable à l ’ eau potable pour tous les...                0   

   chunk_id  
0         1  
1         2  
(2, 8)
['Dateiname', 'Top-Sachgebiet 1', 'Top-Sachgebiet 2', 'Top-Sachgebiet 3', 'Top-3 alle', 'Text', 'original_row_id', 'chunk_id']


In [143]:
# 5. Filtern von Zeilen, um zu überprüfen, wie der Text aufgeteilt wurde

# Filtern nach den ersten 3 unique original_row_ids
first_three_ids = df_new["original_row_id"].unique()[:3]

# Zeilen mit diesen original_row_ids herausfiltern
filtered_df = df_new[df_new["original_row_id"].isin(first_three_ids)]

# Anzeigen der gefilterten Zeilen
print(filtered_df)


                    Dateiname Top-Sachgebiet 1 Top-Sachgebiet 2  \
0  162799BFL_Stagiaire_FR.txt           Andere               HR   
1  162799BFL_Stagiaire_FR.txt           Andere               HR   

           Top-Sachgebiet 3  \
0  Struktur und Institution   
1  Struktur und Institution   

                                          Top-3 alle  \
0  Die Top 3 Sachgebiete in diesem Text sind: And...   
1  Die Top 3 Sachgebiete in diesem Text sind: And...   

                                                Text  original_row_id  \
0  Unnamed : 0 Unnamed : 1 Unnamed : 2 Unnamed : ...                0   
1  ##plémentaire, veuillez - vous adresser à Mme ...                0   

   chunk_id  
0         1  
1         2  


In [56]:
# Step 3: Load Model and do Embeddings

In [162]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ---------- SCHRITT 1: CSV mit Text-Chunks + Topics laden ----------
chunks_df = pd.read_csv(done_csv_name)
chunks = chunks_df["Text"].tolist()

# Nehme die erste Zeile aus "Top-3 alle" (sie ist ja überall gleich)
topics_text = chunks_df["Top-3 alle"].iloc[0]
print(f"📚 {len(chunks)} Chunks geladen.")
print(f"🏷️  Topics-Kontext: {topics_text}")

# ---------- SCHRITT 2: Modell laden & Chunks + Topics embeddieren ----------
model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
chunk_embeddings = model.encode(chunks, show_progress_bar=True)
topics_embedding = model.encode(topics_text)
print("📐 Embeddings erstellt.")

# ---------- SCHRITT 3: Durchschnitts-Vektor + Topics kombinieren (per Concatenate!) ----------
text_embedding = np.mean(chunk_embeddings, axis=0)
print("text_embedding.shape:", text_embedding.shape)        # z.B. (512,)
print("topics_embedding.shape:", topics_embedding.shape)    # z.B. (512,)

# Concatenate wie im Corpus!
query_embedding = np.concatenate([text_embedding, topics_embedding])
print("query_embedding.shape:", query_embedding.shape)      # Sollte (1024,) sein

# Jetzt kannst du wie gewohnt mit den corpus embeddings vergleichen, z.B.:
# similarities = cosine_similarity([query_embedding], corpus_embeddings)[0]


📚 2 Chunks geladen.
🏷️  Topics-Kontext: Die Top 3 Sachgebiete in diesem Text sind: Struktur und Institution, Entwicklung und Zusammenarbeit, Humanitäre Hilfe.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📐 Embeddings erstellt.
text_embedding.shape: (512,)
topics_embedding.shape: (512,)
query_embedding.shape: (1024,)


In [13]:
# Step 4: Load Database 

In [163]:
# ---------- SCHRITT 4: Datenbank laden ----------
embeddings = np.load("embeddings.npy")
metadata = pd.read_csv("metadata.csv")

# ✅ Zwischenausgabe: Prüfen, ob Embeddings & Metadaten korrekt geladen wurden
print(f"📂 Embeddings geladen: Shape {embeddings.shape}")
print(f"📝 Metadaten geladen: {len(metadata)} Einträge")

# Zeige die ersten 3 Zeilen der Metadaten als Vorschau
print("\n🔎 Vorschau der Metadaten:")
print(metadata.head(3))



📂 Embeddings geladen: Shape (6450, 1024)
📝 Metadaten geladen: 6450 Einträge

🔎 Vorschau der Metadaten:
   original_row_id  chunk_id Translator  \
0                0         1        kas   
1                0         2        kas   
2                0         3        kas   

                                          Top-3 alle  
0  Die Top 3 Sachgebiete in diesem Text sind: Ent...  
1  Die Top 3 Sachgebiete in diesem Text sind: Ent...  
2  Die Top 3 Sachgebiete in diesem Text sind: Ent...  


In [ ]:
# Step 5: Give out Translator

In [146]:
print("Query Embedding Shape:", query_embedding.shape)  # Sollte (1024,) sein
print("Corpus Embeddings Shape:", embeddings.shape)     # Sollte (n, 1024) sein


Query Embedding Shape: (1024,)
Corpus Embeddings Shape: (6450, 1024)


In [164]:
# ---------- SCHRITT 5: Ähnlichste Übersetzer:innen finden ----------
similarities = cosine_similarity([query_embedding], embeddings)[0]
best_idx = similarities.argmax()
beste_uebersetzerin = metadata.loc[best_idx, "Translator"]

# ---------- AUSGABE ----------
print("\n✨ Empfohlene Übersetzer:in:", beste_uebersetzerin)
print("🔍 Ähnlichkeitswert:", similarities[best_idx])

# Optional: Top 3 anzeigen
top_n = 3
top_indices = similarities.argsort()[-top_n:][::-1]
print("\nTop 3 Übersetzer:innen:")
for rank, idx in enumerate(top_indices, 1):
    print(f"{rank}. {metadata.loc[idx, 'Translator']} (Ähnlichkeit: {similarities[idx]:.4f})")


✨ Empfohlene Übersetzer:in: bec
🔍 Ähnlichkeitswert: 0.7750152

Top 3 Übersetzer:innen:
1. bec (Ähnlichkeit: 0.7750)
2. bec (Ähnlichkeit: 0.7562)
3. bec (Ähnlichkeit: 0.7508)


In [ ]:
# Gewichtung anpassen: 
# query_embedding = 0.7 * text_embedding + 0.3 * topics_embedding
# ➡️ Beispiel: Wenn du willst, dass die Topics stärker zählen, erhöhe z. B. auf 0.5 * text_embedding + 0.5 * topics_embedding.